In [1]:
!pip install kaggle

     ---------------------------------------- 0.0/82.1 kB ? eta -:--:--
     --------- ---------------------------- 20.5/82.1 kB 330.3 kB/s eta 0:00:01
     ------------------ ------------------- 41.0/82.1 kB 393.8 kB/s eta 0:00:01
     -------------------------------------  81.9/82.1 kB 657.6 kB/s eta 0:00:01
     -------------------------------------- 82.1/82.1 kB 509.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
   ---------------------------------------- 0.0/78.2 kB ? eta -:--:--
   ---------------------------------------- 78.2/78.2 kB 2.1 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105128 sha256=aa565369192a678c1b0fe4b7aa4402f820a9953f283f1a1f1b4b4f7a20fd5eac
  Stored in directory: c:\users\mehardeep kaur\appdata\local\pip\cache\wheels\89\bc\52\0d140fc1

In [7]:
!kaggle datasets list -s "email"

ref                                                           title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
wcukierski/enron-email-dataset                                The Enron Email Dataset                           358MB  2016-06-16 20:55:19          49396        705  0.7647059        
ashfakyeafi/spam-email-classification                         Spam email classification                         207KB  2023-12-22 19:59:51           1917         88  1.0              
mfaisalqureshi/spam-email                                     Spam Email                                        207KB  2021-06-21 08:16:42           8363         54  0.7647059        
subhajournal/phishingemails                                   Phishing Email Det

In [9]:
!kaggle datasets download -d "wcukierski/enron-email-dataset"

Dataset URL: https://www.kaggle.com/datasets/wcukierski/enron-email-dataset
License(s): copyright-authors




  0%|          | 0.00/358M [00:00<?, ?B/s]
  0%|          | 1.00M/358M [00:01<09:48, 636kB/s]
  1%|          | 2.00M/358M [00:01<04:52, 1.28MB/s]
  1%|          | 3.00M/358M [00:02<03:01, 2.05MB/s]
  1%|1         | 4.00M/358M [00:02<02:04, 2.97MB/s]
  1%|1         | 5.00M/358M [00:02<01:32, 4.00MB/s]
  2%|1         | 6.00M/358M [00:02<01:12, 5.06MB/s]
  2%|2         | 8.00M/358M [00:02<00:54, 6.72MB/s]
  3%|2         | 9.00M/358M [00:02<00:50, 7.18MB/s]
  3%|3         | 11.0M/358M [00:02<00:44, 8.26MB/s]
  3%|3         | 12.0M/358M [00:02<00:42, 8.51MB/s]
  4%|3         | 13.0M/358M [00:03<00:41, 8.74MB/s]
  4%|4         | 15.0M/358M [00:03<00:39, 9.06MB/s]
  4%|4         | 16.0M/358M [00:03<00:39, 9.09MB/s]
  5%|4         | 17.0M/358M [00:03<00:42, 8.36MB/s]
  5%|5         | 19.0M/358M [00:03<00:38, 9.35MB/s]
  6%|5         | 21.0M/358M [00:03<00:35, 10.0MB/s]
  6%|6         | 23.0M/358M [00:04<00:32, 10.8MB/s]
  7%|6         | 25.0M/358M [00:04<00:31, 11.2MB/s]
  8%|7         | 27.0

In [13]:
import zipfile
with zipfile.ZipFile('enron-email-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('email_classification/enron-email-dataset')

In [17]:
!pip install pandas

  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [18]:
import pandas as pd

In [21]:
email_df = pd.read_csv("emails.csv")


In [22]:
email_df

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...


In [43]:
row_1 = email_df.iloc[45]
row_1[1]

C:\Users\Mehardeep Kaur\AppData\Local\Temp\ipykernel_22080\142348224.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_1[1]


"Message-ID: <28779445.1075855688226.JavaMail.evans@thyme>\nDate: Wed, 13 Sep 2000 06:02:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: stagecoachmama@hotmail.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: stagecoachmama@hotmail.com\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail\nX-Origin: Allen-P\nX-FileName: pallen.nsf\n\nLucy,\n\nI want to have an accurate rent roll as soon as possible. I faxed you a copy \nof this file.  You can fill in on the computer or just write in the correct \namounts and I will input.\n"

In [45]:
email_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


no, null values, both are of string datatypes
structure --

1. check if their are same messages or not

In [50]:
email_df.message.nunique()

517401